# Spot/short grid search example

See the earlier notebook for more information.

### Grid search target strategy loading


In [ ]:
import datetime
import importlib.util
import sys

from tradeexecutor.utils.default_strategies import get_default_strategies_path

strategies_folder = get_default_strategies_path()
strategy_path = strategies_folder / "spot-and-short-momentum-grid.py"

# See https://stackoverflow.com/a/67692/315168
spec = importlib.util.spec_from_file_location("strategy_mod", strategy_path)
python_mod = importlib.util.module_from_spec(spec)
sys.modules["strategy_mod"] = python_mod
spec.loader.exec_module(python_mod)

print(f"Backtesting for strategy {strategy_path}, engine version {python_mod.trading_strategy_engine_version}")

### Set up the market data client



In [ ]:
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

Load the strategy universe and analyse what trading pairs the strategy code gives for us.

In [ ]:
from tradeexecutor.strategy.execution_context import notebook_execution_context

universe = python_mod.create_trading_universe(
    datetime.datetime.utcnow(),
    client,
    notebook_execution_context,
    python_mod.universe_options,
)

## Available trading universe

Inspect trading pairs and lending markets we are going to backtest.


In [ ]:
from IPython.display import HTML

from tradeexecutor.analysis.universe import analyse_long_short_universe
from tradingstrategy.utils.jupyter import format_links_for_html_output

df = analyse_long_short_universe(universe)
df = format_links_for_html_output(df, ("Price data page", "Lending rate page",))

display(HTML(df.to_html(escape=False)))

## Run grid search

In [ ]:
from pathlib import Path
from tradeexecutor.backtest.grid_search import perform_grid_search, prepare_grid_combinations

# This is the path where we keep the result files around
storage_folder = Path(f"/tmp/{strategy_path.stem}-grid-search")

parameters = {
    "cycle_duration_days": [6, 7, 8],
    "momentum_lookback": [6, 7, 8],
    "take_profit": [1.06, 1.07],
    "negative_take_profit": [1.06, 1.07],
    "positive_mometum_threshold": [0.0001, 0.025],
    "negative_mometum_threshold": [-0.06],
}
combinations = prepare_grid_combinations(parameters, storage_folder, clear_cached_results=True)

grid_search_results = perform_grid_search(
    python_mod.grid_search_worker,
    universe,
    combinations,
    max_workers=4,
)


## Examine grid search results


In [ ]:
from tradeexecutor.analysis.grid_search import analyse_grid_search_result
from tradeexecutor.analysis.grid_search import visualise_table
table = analyse_grid_search_result(grid_search_results)

visualise_table(table)